### The notebook helps you to run a simulation of the cloud compute costs for using an autoscaling Jupyter hub deployment.

In [ ]:
# What allows us to make changes to the z2jh_cost_simulator package
# and workaround a caching of the module, so our changes can be seen.
%load_ext autoreload
%autoreload 2

In [ ]:
#Import the simulator.py module from the z2jh_cost_simulator package
from z2jh_cost_simulator import simulator
from z2jh_cost_simulator import generate_user_activity

## Set the configurations for the simulation.
1. Info about a single node pool where users reside
    - CPU / Memory, for example 4 CPU cores / 26 GB memory node
    - Autoscaling limits, for example 0-5 nodes
    - Cost, for example 120 USD / month and node
    - Cluster autoscaler details, how long of node inactivity is required(10 minutes) 
1. Info about the resource requests (guaranteed resources) for the users
1. How much time before a user pod is culled by inactivity 
1. The  max lifetime for the user pod before it can be culled down.


In [ ]:
import ipywidgets as widgets
from ipywidgets import Layout, VBox


#to see the alignment of the sliders.
style = {'description_width': 'initial'}
box_layout = Layout(border='solid',
                    width='50%')
heading = widgets.HTML(value="<b>Select the simulation configuration.</b>")
max_min_nodes = widgets.IntRangeSlider(min=1,
                                      max=10,
                                      step=1,
                                      description='Number of nodes:',
                                      disabled=False,
                                      continuous_update=False,
                                      orientation='horizontal',
                                      readout=True,
                                      style=style)
node_cpu = widgets.IntSlider(min=1,max=16,description="Node CPU",style=style)
node_memory = widgets.FloatSlider(min=0,max=128.0,step =1,description="Node Memory(in GB)",style=style)
user_pod_cpu = widgets.FloatSlider(min=0,max=5,step =.05, description="User pod CPU",style=style)
user_pod_memory = widgets.FloatSlider(min=0,max=5000.0,step=64,description="User pod Memory(in MB)",style=style)
cost_per_month = widgets.FloatSlider(min=0,max=150.0,step=5.5,description="Cost per month",style=style)
pod_culling_max_inactivity_time = widgets.IntSlider(min=1,max=100,step=5,description="Pod culling for inactivity",style=style)
pod_culling_max_lifetime = widgets.IntSlider(min=1,max=300,step=20,description="Pod culling for max lifetime",style=style)
node_configurations = [heading,max_min_nodes,node_cpu,user_pod_cpu,node_memory,user_pod_memory,cost_per_month,pod_culling_max_inactivity_time,pod_culling_max_lifetime ]

VBox(node_configurations,layout = box_layout)

In [ ]:
configurations = {'min_nodes': 1,'max_nodes':3,
                                'node_cpu': 4 ,'node_memory': 15,
                                'user_pod_cpu':.04,'user_pod_memory': 5,
                                'cost_per_month': 12.8,
                                'pod_inactivity_time':3,
                                'pod_max_lifetime':7 ,'node_stop_time':5}


In [ ]:
#create an object of the Simulation, pass the configurations and the user activity to the simulation.
# we will be calling the function to generate the user activity and pass it to the Simulation object.
hour_wise_users = [2,3,4,2,3,4,5,5]

user_activity = generate_user_activity.generate_user_activity(hour_wise_users)
sim = simulator.Simulation(configurations, user_activity)
#sim.add_users()
#The run_simulation logic runs the simulation for one week. 
sim.run_simulation(stop=len(hour_wise_users)*60)
cluster_utilization_data = sim.create_utilization_data()

### The line chart helps you to visualize the utilization of the cluster nodes and the cluster auto scaling for one week.

In [ ]:
#imports required for the visualization of the simulation 
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode , plot, iplot

from ipywidgets import Layout, VBox
style = {'description_width': 'initial'}
day_selected = widgets.IntSlider(min=1,max=7,description="Day of the week",style=style)
list_nodes = list(node for node in cluster_utilization_data.columns if node.find('percent') != -1)
data = []
for node in list_nodes:
    data.append(
        go.Scatter(
                x = cluster_utilization_data['time'],
                y = cluster_utilization_data[node][0:1440],
                mode = 'lines',
                name = 'Node ' + str(list_nodes.index(node)+1)
                  ))

g = go.FigureWidget(data=data,
                    layout=go.Layout(
                        title=dict(
                            text='Cluster utilization Data'
                        ),
                       xaxis=dict(
                            title='time in minutes',
                            #tickmode='linear',
                            #tick0=0,
                            #dtick=60,
                            tickmode = 'array',
                            tickangle=45,
                            tickvals = list(range(0,10080,120)),
                            ticktext = [str(i) + " hours" for i in range(0,24,2)]
                        ),
                      yaxis=dict(
                        title='utilization(%)',
                        tickformat="%"
                        
                        ) 
                    ))

def response(change):
    for node in list_nodes:
        x1 = cluster_utilization_data[node][(day_selected.value-1)*1440:day_selected.value*1440]
        g.data[list_nodes.index(node)].y = x1
day_selected.observe(response)
container2 = widgets.HBox([day_selected])
widgets.VBox([
            container2,g
                    ])       

In [ ]:
#for 200 days
user_activity = []
for no_users in range(5):
    #user_obj = User()
    activity = []
    user_activity_array = np.array(sorted(np.random.choice([0,1], size=50, p=[.4, .6])))
    for i in range(4):
        activity = np.concatenate((activity,user_activity_array),axis=None)
    user_activity.append(activity)

In [ ]:
#imports required for the visualization of the simulation

#create an object of the Simulation, pass the configurations and the user activity to the simulation.
sim = simulator.Simulation(configurations,user_activity)

#The run_simulation logic runs the simulation for one week. 
sim.run_simulation(stop=200)
cluster_utilization_data = sim.create_utilization_data()

In [ ]:
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode , plot, iplot
style = {'description_width': 'initial'}
time = widgets.IntSlider(min=1,max=10,description="Range in minutes",style=style)
list_nodes = list(col for col in cluster_utilization_data.columns if col.find('percent') != -1)
data = []
for node in list_nodes:
    data.append(
        go.Scatter(
                x = cluster_utilization_data['time'],
                y = cluster_utilization_data[node],
                mode = 'lines',
                name = 'Node' + str(list_nodes.index(node)+1)
                         ))

g = go.FigureWidget(data=data,
                    layout=go.Layout(
                        title=dict(
                            text='utilization data'
                        ),
                       xaxis=dict(
                            title='time in minutes',
                            tickmode='linear',
                            tick0=0,
                            dtick=1
                           #tickmode = 'array',
                           #tickvals = [1, 3, 5, 7, 9, 11],
                           #ticktext = ['One', 'Three', 'Five', 'Seven', 'Nine', 'Eleven']
                        ),
                      yaxis=dict(
                        title='utilization(%)'
                        ) 
                    ))

def response(change):
    for node in list_nodes:
        x1 = cluster_utilization_data[node][(time.value-1)*20:time.value*20]
        g.data[list_nodes.index(node)].y = x1
time.observe(response)
container2 = widgets.HBox([time])
widgets.VBox([
            container2,g
            ])       
            